In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import os

from src.data_processing import load_config, load_data, preprocess_data, sample_data

# Configuration visuelle
sns.set(style="whitegrid", context="talk")
%matplotlib inline

# Charger la configuration
config = load_config("config/config.yaml")
print("Configuration chargée avec succès.")


💾 Cellule 2 — Chargement du dataset complet depuis BigQuery
python
Copier
Modifier


In [ ]:
from pandas_gbq import read_gbq

# Récupération du project_id depuis la configuration
project_id = config.get("project_id", "avisia-certification-ml-yde")

# Requête SQL pour charger TOUT le dataset (attention : très volumineux >187M lignes)
query = """
SELECT
  trip_start_timestamp,
  pickup_community_area
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE pickup_community_area IS NOT NULL
"""

print("Chargement des données depuis BigQuery... (peut prendre plusieurs minutes)")

# Chargement via pandas-gbq
df_raw = read_gbq(query, project_id=project_id)

print(f"✅ Dataset chargé avec succès. Nombre de lignes : {len(df_raw):,}")
df_raw.head()


⏰ Cellule 3 — Agrégation par heure et zone (trip_count par timestamp_hour × pickup_community_area)

# Convertir les timestamps au format horaire (arrondi à l'heure)
df_raw["timestamp_hour"] = pd.to_datetime(df_raw["trip_start_timestamp"]).dt.floor("H")

# Agrégation : nombre de courses par heure et par pickup_community_area
df_demand = (
    df_raw
    .groupby(["timestamp_hour", "pickup_community_area"])
    .size()
    .reset_index(name="trip_count")
    .sort_values(["timestamp_hour", "pickup_community_area"])
)

print(f"✅ Données agrégées : {len(df_demand):,} lignes.")
df_demand.head()


🧩 Cellule 4 — Complétion des séries temporelles (remplir les heures sans données)

In [ ]:
# %% [code]
# Créer l'ensemble des heures disponibles dans le dataset (par pas de 1 heure)
min_time = df_demand["timestamp_hour"].min()
max_time = df_demand["timestamp_hour"].max()
all_hours = pd.date_range(start=min_time, end=max_time, freq="H")

# Identifier toutes les zones uniques
all_zones = df_demand["pickup_community_area"].dropna().unique()
all_zones = sorted(all_zones)

# Créer le produit cartésien : toutes les combinaisons heure × zone
complete_index = pd.MultiIndex.from_product(
    [all_hours, all_zones],
    names=["timestamp_hour", "pickup_community_area"]
)

# Créer un DataFrame complet
df_complete = pd.DataFrame(index=complete_index).reset_index()

# Fusionner avec les données observées
df_demand_complete = pd.merge(
    df_complete,
    df_demand,
    on=["timestamp_hour", "pickup_community_area"],
    how="left"
)

# Remplacer les valeurs manquantes (heures sans courses) par 0
df_demand_complete["trip_count"] = df_demand_complete["trip_count"].fillna(0).astype(int)

print(f"✅ Séries temporelles complétées : {len(df_demand_complete):,} lignes.")
df_demand_complete.head()


🧠 Cellule 5 — Ajout des features temporelles (hour, day, month, etc.)


In [ ]:
# %% [code]
# Extraire les features temporelles classiques
df_demand_complete["hour"] = df_demand_complete["timestamp_hour"].dt.hour
df_demand_complete["day_of_week"] = df_demand_complete["timestamp_hour"].dt.dayofweek  # Lundi = 0
df_demand_complete["month"] = df_demand_complete["timestamp_hour"].dt.month
df_demand_complete["day_of_year"] = df_demand_complete["timestamp_hour"].dt.dayofyear
df_demand_complete["week_of_year"] = df_demand_complete["timestamp_hour"].dt.isocalendar().week.astype(int)
df_demand_complete["year"] = df_demand_complete["timestamp_hour"].dt.year
df_demand_complete["is_weekend"] = df_demand_complete["day_of_week"].isin([5, 6]).astype(int)

# Optionnel : encodage cyclique (utile pour XGBoost, moins pour AutoML Forecasting qui encode en interne)
df_demand_complete["hour_sin"] = np.sin(2 * np.pi * df_demand_complete["hour"] / 24)
df_demand_complete["hour_cos"] = np.cos(2 * np.pi * df_demand_complete["hour"] / 24)

print("✅ Features temporelles ajoutées.")
df_demand_complete[["timestamp_hour", "hour", "day_of_week", "month", "is_weekend"]].head()


💾 Cellule 6 — Export des données finales vers BigQuery


In [ ]:
# %% [code]
from google.cloud import bigquery

# Initialisation du client BigQuery
client = bigquery.Client(project="avisia-certification-ml-yde")

# Définir la destination
table_id = "avisia-certification-ml-yde.chicago_taxis.demand_by_hour"

# Option : supprimer les colonnes inutiles ou spécifiques à XGBoost
columns_to_keep = [
    "timestamp_hour", "pickup_community_area", "trip_count",
    "hour", "day_of_week", "month", "year", "day_of_year", "week_of_year", "is_weekend"
]

# On sélectionne uniquement les colonnes utiles pour Vertex AI Forecast
df_to_export = df_demand_complete[columns_to_keep]

# Écriture dans BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

job = client.load_table_from_dataframe(
    df_to_export, table_id, job_config=job_config
)

job.result()  # Attendre la fin du job

print(f"✅ Table exportée vers BigQuery : {table_id}")
